In [53]:
import os
import pandas as pd

directory = "MIDTERM23_24Summer"
subdirectory = "data for task 1"
documents = ["CNC", "EQR", "IVZ", "TRV"]

df = pd.DataFrame()

for doc in documents:
    filename = doc + ".csv"
    path = os.path.join(directory, subdirectory, filename)

    with open(path, "r") as f:
        data_full = pd.read_csv(f)

        columns = data_full.columns
        column_dict = {}
        for col in columns:
            column_dict[col] = f"{doc} {col}"
        column_dict['Date'] = 'Date'

        data_filt = data_full.rename(columns=column_dict)
        if df.empty:
            df = data_filt
        else:
            df = df.merge(data_filt, on="Date", how="outer")

In [84]:
df.head()

,Date,CNC High,CNC Low,CNC Open,CNC Close,CNC Volume,CNC Adj Close,EQR High,EQR Low,EQR Open,...,IVZ Open,IVZ Close,IVZ Volume,IVZ Adj Close,TRV High,TRV Low,TRV Open,TRV Close,TRV Volume,TRV Adj Close
0,2017-11-13,47.494999,46.555000,46.555000,47.180000,3056200.0,47.180000,70.250000,68.800003,69.150002,...,34.630001,34.520000,1896400.0,27.556639,135.440002,133.199997,135.160004,133.320007,2742700.0,120.533607
1,2017-11-14,47.180000,46.375000,46.974998,46.514999,1715400.0,46.514999,70.349998,69.570000,70.089996,...,34.290001,34.470001,2247800.0,27.516727,133.979996,131.690002,132.850006,133.860001,1812200.0,121.021782
2,2017-11-15,46.509998,45.035000,46.400002,46.075001,2552800.0,46.075001,70.089996,68.900002,69.970001,...,34.209999,34.480000,4631800.0,27.524712,134.199997,132.750000,133.119995,133.710007,1837900.0,120.886192
3,2017-11-16,48.465000,46.255001,46.255001,47.970001,3466600.0,47.970001,69.419998,68.529999,69.070000,...,34.709999,34.570000,2280100.0,27.596558,133.559998,130.660004,133.380005,130.809998,2375800.0,118.264328
4,2017-11-17,47.974998,47.520000,47.834999,47.570000,2009200.0,47.570000,69.300003,68.169998,68.980003,...,34.430000,34.980000,2899300.0,27.923851,130.789993,129.720001,130.669998,129.919998,2372100.0,117.459679


In [87]:
df.head()
type(df['Date'][0])

df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True) 

In [66]:
for doc in documents:
    bool = (df[f'{doc} Close'] == df[f'{doc} Adj Close']).all()
    print(f"{doc}: {bool}")

CNC: True
EQR: False
IVZ: False
TRV: False


In [69]:
for doc in documents:
    max_val = df[f'{doc} Adj Close'].max()
    min_val = df[f'{doc} Adj Close'].min()
    print(doc, '- max :', max_val, ', min :', min_val)

CNC - max : 97.22000122070312 , min : 42.72999954223633
EQR - max : 93.23999786376952 , min : 44.40938949584961
IVZ - max : 30.65396881103516 , min : 6.301447868347168
TRV - max : 187.75 , min : 78.2640151977539


In [100]:
import numpy as np
import pandas as pd

# Initialize an empty DataFrame to store all log returns
log_returns = pd.DataFrame()

for doc in documents:
    # Calculate log difference for the current document
    log_diff = np.log(df[f'{doc} Adj Close']).diff()
    
    # Add the log_diff Series to the DataFrame
    if log_returns.empty:
        # For the first document, create the DataFrame with the first column
        log_returns[doc] = log_diff
    else:
        # For subsequent documents, add as a new column
        log_returns[doc] = log_diff
    
    # Calculate summary statistics
    summary = log_diff.aggregate(['min', 'max', 'mean', 'median'])
    print(doc)
    print(summary)
    print('highest gain', log_diff.idxmax())
    print('highest loss', log_diff.idxmin())

CNC
min      -0.162602
max       0.144987
mean      0.000440
median    0.000750
Name: CNC Adj Close, dtype: float64
highest gain 2020-03-04 00:00:00
highest loss 2020-03-16 00:00:00
EQR
min      -0.185287
max       0.140302
mean      0.000025
median    0.000599
Name: EQR Adj Close, dtype: float64
highest gain 2020-11-09 00:00:00
highest loss 2020-03-16 00:00:00
IVZ
min      -0.237406
max       0.164246
mean     -0.000264
median    0.000780
Name: IVZ Adj Close, dtype: float64
highest gain 2022-11-10 00:00:00
highest loss 2020-04-23 00:00:00
TRV
min      -0.233199
max       0.124783
mean      0.000326
median    0.001302
Name: TRV Adj Close, dtype: float64
highest gain 2020-03-17 00:00:00
highest loss 2020-03-16 00:00:00


In [91]:
for doc in documents:
    weekly_volume = df[f'{doc} Volume'].resample('W').sum()
    print(weekly_volume.mean())

19492156.32183908
9481804.597701149
23875980.842911877
7120086.206896552


In [105]:
for doc in documents:
    diff_return = df[f"{doc} Adj Close"][len(df) - 1] - df[f"{doc} Adj Close"][0]
    total_log_returns = log_returns[doc].sum()
    cumulative_returns = np.exp(total_log_returns) - 1
    print(cumulative_returns)
    print(diff_return)

0.7395082484944562
34.88999938964844
0.03203663092283349
1.9488258361816477
-0.28256850698194824
-7.786638259887695
0.5077122432704562
61.19638824462888


C:\Users\svest\AppData\Local\Temp\ipykernel_21520\2162791637.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  diff_return = df[f"{doc} Adj Close"][len(df) - 1] - df[f"{doc} Adj Close"][0]


In [ ]:
import matplotlib.pyplot as plt

plt.subplots(1,1, figsize = (20,7))
for doc in documents:
    